# Desafio 2 - Regressão: Treinamento de Modelo

## Carregamento de features em diferentes formatos e target

In [1]:
import pickle
file_path = '/content/drive/MyDrive/curso_machine-learning-python/datasets/proccessed_insurance_data.pkl'
processed_insurance_data = open(file_path, 'rb')
target = pickle.load(processed_insurance_data)
all_features = pickle.load(processed_insurance_data)
scaled_all_features = pickle.load(processed_insurance_data)
high_corr_features = pickle.load(processed_insurance_data)
scaled_high_corr_features = pickle.load(processed_insurance_data)
processed_insurance_data.close()

In [2]:
import numpy as np
import pandas as pd

## Verificação de dois dos conjuntos carregados

In [3]:
target_df = pd.DataFrame(target)
target_df.shape

(1338, 1)

In [4]:
all_features_df = pd.DataFrame(all_features)
all_features_df.shape

(1338, 11)

## Separação de dados em treino e teste

In [5]:
def divide_dataset_treino_teste(source_features, source_target):
  from sklearn.model_selection import train_test_split
  features_treino, features_teste, target_treino, target_teste = train_test_split(
    source_features, source_target, test_size = 0.3, random_state = 0
  )
  return features_treino, features_teste, target_treino, target_teste

In [307]:
features_treino, features_teste, target_treino, target_teste = divide_dataset_treino_teste(all_features, target)

## Treinamento de modelos

### Multiple Linear Regression

In [7]:
from sklearn.linear_model import LinearRegression

In [8]:
lin_reg_model = LinearRegression()

In [9]:
lin_reg_model.fit(features_treino, target_treino)

LinearRegression()

In [10]:
lin_reg_model.score(features_treino, target_treino)

0.7309569871174701

In [11]:
lin_reg_model.score(features_teste, target_teste)

0.7909160991789905

#### Resultados

**Modelo:** Regressão Linear Múltipla

---

**Conjunto de features:** all_features

**score em treino:** 0.73

**score em teste:** 0.79

---

**Conjunto de features:** scaled_all_features

**score em treino:** 0.73

**score em teste:** 0.79

---


**Conjunto de features:** high_corr_features

**score em treino:** 0.70

**score em teste:** 0.76

---


**Conjunto de features:** scaled_high_corr_features

**score em treino:** 0.70

**score em teste:** 0.76



As features possuem muitas dimensões, não vai ser possível fazer um gráfico com a reta de regressão.

#### Métricas de Desempenho

In [12]:
from sklearn.metrics import root_mean_squared_error

In [13]:
predicted_target = lin_reg_model.predict(features_teste)
print(f"Erro médio usando all_features: {root_mean_squared_error(target_teste, predicted_target)}")

Erro médio usando all_features: 5774.2963057808665


#### Validação Cruzada

In [14]:
from sklearn.model_selection import KFold, cross_val_score

In [15]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [16]:
modelo = LinearRegression()

In [17]:
resultado = cross_val_score(modelo, all_features, target, cv = k_fold)

In [18]:
print(f"Coeficiente R² médio: {resultado.mean()}")
print(f"Coeficiente R² inferior: {resultado.mean() - resultado.std()}")
print(f"Coeficiente R² superior: {resultado.mean() + resultado.std()}")

Coeficiente R² médio: 0.7356373982088413
Coeficiente R² inferior: 0.680117796174853
Coeficiente R² superior: 0.7911570002428296


#### Conclusão

Modelo baseado em Regressão Linear Múltipla conseguiu se ajustar em 79% à target. É um dos modelos mais simples, então parece um bom resultado. Vamos testar outros modelos em seguida.

### Support Vector Regressor

In [19]:
from sklearn.svm import SVR

In [20]:
# default kernel is rbf
sup_vec_regressor = SVR(kernel = 'rbf')

In [21]:
sup_vec_regressor.fit(features_treino, target_treino)

SVR()

In [22]:
sup_vec_regressor.score(features_treino, target_treino)

-0.09426391963281566

In [23]:
sup_vec_regressor.score(features_teste, target_teste)

-0.08859789219262182

Do que o Support Vector Regressor precisa mesmo? Padronizar escalas de variáveis, tanto dependentes quando independentes.

In [24]:
from sklearn.preprocessing import StandardScaler
feature_scaler = StandardScaler()
scaled_features_treino = feature_scaler.fit_transform(features_treino)

In [25]:
target_scaler = StandardScaler()
scaled_target_treino = target_scaler.fit_transform(target_treino.reshape(-1, 1))

In [26]:
SVR_wscaled = SVR(kernel = 'rbf')
SVR_wscaled.fit(scaled_features_treino, scaled_target_treino)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [27]:
SVR_wscaled.score(scaled_features_treino, scaled_target_treino)

0.8374931487203512

In [28]:
scaled_features_teste = feature_scaler.fit_transform(features_teste)

In [29]:
scaled_target_teste = target_scaler.fit_transform(target_teste.reshape(-1,1))

In [30]:
SVR_wscaled.score(scaled_features_teste, scaled_target_teste)

0.8752604597810801

O coeficiente de determinação $R^2$ ficou melhor no SVR.

#### Resultados

**Modelo:** Support Vector Regressor

---

**Conjunto de features:** scaled_all_features and scaled_target

**score em treino:** 0.84

**score em teste:** 0.88


#### Métricas de Desempenho

In [31]:
from sklearn.metrics import root_mean_squared_error

In [32]:
predicted_target = SVR_wscaled.predict(scaled_features_teste)

In [33]:
target_teste_inversed = target_scaler.inverse_transform(scaled_target_teste)

In [34]:
predicted_target_inversed = target_scaler.inverse_transform(predicted_target.reshape(-1, 1))

In [35]:
root_mean_squared_error(target_teste_inversed, predicted_target_inversed)

4460.061407282226

#### Validação Cruzada

In [36]:
from sklearn.model_selection import KFold, cross_val_score

In [37]:
X_scaler = StandardScaler()

In [38]:
y_scaler = StandardScaler()

In [39]:
X_scaled = X_scaler.fit_transform(all_features)

In [40]:
y_scaled = y_scaler.fit_transform(target.reshape(-1, 1))

In [41]:
modelo = SVR(kernel = 'rbf')

In [42]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [43]:
resultado = cross_val_score(modelo, X_scaled, y_scaled.ravel(), cv = k_fold)

In [44]:
print(f"Coeficiente R² médio: {resultado.mean()}")
print(f"Coeficiente R² inferior: {resultado.mean() - resultado.std()}")
print(f"Coeficiente R² superior: {resultado.mean() + resultado.std()}")

Coeficiente R² médio: 0.8356454051983156
Coeficiente R² inferior: 0.7880730379941618
Coeficiente R² superior: 0.8832177724024695


#### Conclusão

O regressor baseado em support vector machines se saiu melhor do que a regressão linear, tanto no coeficiente R² quanto na métrica de erro RMSE.
Entre os 2, deve ser escolhido SVR. Apesar disso, o contra do SVR é que exige várias transformações de escala para poder ser utilizado. Talvez por isso, muitas vezes as pessoas escolhem outros modelos.

### Decision Tree Regressor

In [45]:
from sklearn.tree import DecisionTreeRegressor

In [46]:
tree = DecisionTreeRegressor(max_depth = 4, criterion='poisson', random_state = 0)

In [47]:
tree.fit(features_treino, target_treino)

DecisionTreeRegressor(criterion='poisson', max_depth=4, random_state=0)

In [48]:
tree.score(features_treino, target_treino)

0.8531650151210386

In [49]:
tree.score(features_teste, target_teste)

0.8854782318253775

squared_error:
- treino: 0.852 ~ 0.85
- teste: 0.884 ~ 0.88

poisson:
- treino: 0.853 ~ 0.85
- teste: 0.885 ~ 0.89

#### Resultados

all_features == scaled_all_features

R² em treino: 0.85

R² em teste: 0.89

#### Métricas de Desempenho

In [50]:
from sklearn.metrics import root_mean_squared_error

In [51]:
predicted_target_teste = tree.predict(features_teste)
root_mean_squared_error(target_teste, predicted_target_teste)

4273.490975094296

#### Validação Cruzada

In [52]:
from sklearn.model_selection import KFold, cross_val_score

In [53]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [54]:
modelo = DecisionTreeRegressor(max_depth = 4, criterion='poisson', random_state = 0)

In [55]:
resultado = cross_val_score(modelo, all_features, target)

In [56]:
print(f"Coeficiente R² médio: {resultado.mean()}")
print(f"Coeficiente R² inferior: {resultado.mean() - resultado.std()}")
print(f"Coeficiente R² superior: {resultado.mean() + resultado.std()}")

Coeficiente R² médio: 0.8510140323464425
Coeficiente R² inferior: 0.8141620277296231
Coeficiente R² superior: 0.8878660369632618


#### Conclusão

O Regressor baseado em árvore de decisão superou o Suppor Vector Regressor. A diferença foi pequena, mas ele obteve melhores coeficiente R² além de menor erro com RSME. Além disso, ainda tem a vantagem de exigir menos pré/pós-processamento.

### Random Forest

In [57]:
from sklearn.ensemble import RandomForestRegressor

In [58]:
random_forest = RandomForestRegressor(n_estimators = 100, criterion = 'friedman_mse', max_depth = 6, random_state = 0)

In [59]:
random_forest.fit(features_treino, target_treino)

RandomForestRegressor(criterion='friedman_mse', max_depth=6, random_state=0)

In [60]:
random_forest.score(features_treino, target_treino)

0.9047018875232218

In [61]:
random_forest.score(features_teste, target_teste)

0.8798194991833832

#### Resultados

all_features == scaled_all_features

Random Forest

- score em treino: 0.90

- score em teste: 0.88

#### Métricas de Desempenho

In [62]:
from sklearn.metrics import root_mean_squared_error

In [63]:
predicted_target_teste = random_forest.predict(features_teste)
root_mean_squared_error(target_teste, predicted_target_teste)

4377.798554276961

In [64]:
# Observação: às vezes o algoritmo pode ter um bom score e ter um RMSE ruim. Mudei
# o n_estimators de 250 para 100 para diminuir esse erro. Me parece que estava tendendo ao
# overfitting.

#### Validação Cruzada

In [152]:
from sklearn.model_selection import KFold, cross_val_score

In [153]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [154]:
modelo = RandomForestRegressor(n_estimators = 100, criterion = 'friedman_mse', max_depth = 6, random_state = 0)

In [155]:
resultado = cross_val_score(modelo, all_features, target, cv = k_fold)

In [156]:
print(f"Coeficiente R² médio: {resultado.mean()}")
print(f"Coeficiente R² inferior: {resultado.mean() - resultado.std()}")
print(f"Coeficiente R² superior: {resultado.mean() + resultado.std()}")

Coeficiente R² médio: 0.8502056088233102
Coeficiente R² inferior: 0.807442464238317
Coeficiente R² superior: 0.8929687534083034


#### Conclusão

Random Forest ficou muito parecido com o Decision Tree e ambos foram melhores que o SVR.
Devido à simplicidade, acredito que escolheria o Decision Tree até o momento.
Nas próximas seções, vamos ver 3 dos algoritmos mais poderosos, XGBoost, LightGBM e CatBoost.


### XGBoost

In [163]:
from xgboost import XGBRegressor

In [164]:
xgboost = XGBRegressor(n_estimators = 75, max_depth = 4, learning_rate = 0.05, objective='reg:squarederror')

In [165]:
xgboost.fit(features_treino, target_treino)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=75, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [166]:
xgboost.score(features_treino, target_treino)

0.8813457947769354

In [167]:
xgboost.score(features_teste, target_teste)

0.8901612137439361

#### Resultados

all_features == scaled_all_features

Score do XGBoost:

- treino: 0.88

- teste: 0.89

#### Métricas de Desempenho

In [168]:
from sklearn.metrics import root_mean_squared_error

In [169]:
predicted_target_testes = xgboost.predict(features_teste)

In [170]:
root_mean_squared_error(target_teste, predicted_target_testes)

4185.203996526568

#### Validação Cruzada

In [171]:
from sklearn.model_selection import KFold, cross_val_score

In [172]:
k_fold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [173]:
modelo = XGBRegressor(n_estimators = 75, max_depth = 4, learning_rate = 0.05, objective='reg:squarederror')

In [174]:
cv_score = cross_val_score(modelo, all_features, target, cv = k_fold)

In [175]:
print(f"Coeficiente R² médio: {cv_score.mean()}")
print(f"Coeficiente R² inferior: {cv_score.mean() - cv_score.std()}")
print(f"Coeficiente R² superior: {cv_score.mean() + cv_score.std()}")

Coeficiente R² médio: 0.8577180490554157
Coeficiente R² inferior: 0.8180795485226553
Coeficiente R² superior: 0.8973565495881761


#### Conclusão

O XGBoost apresentou resultados levemente superiores ao outros que estavam melhor no coeficiente de determinação R².
Com relação aos erros, também. O XGBoost apresenta até agora o menor erro, 4185.20.

### LightGBM

In [231]:
import lightgbm as lgb

In [232]:
lgbm = lgb.LGBMRegressor(num_leaves = 15, max_depth = 3, learning_rate = 0.1, n_estimators = 60)

In [233]:
lgbm.fit(features_treino, target_treino)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000106 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 334
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 11
[LightGBM] [Info] Start training from score 13232.916456
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LGBMRegressor(max_depth=3, n_estimators=60, num_leaves=15)

In [234]:
lgbm.score(features_treino, target_treino)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


0.8733899495103583

In [235]:
lgbm.score(features_teste, target_teste)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


0.8970952471386653

#### Resultados

O LightGBM ficou melhor usando scaled_all_features em vez de all_features.
O resultado foi:

- score treino: 0.873 ~ 0.87

- score teste: 0.897 ~ 0.90

#### Métricas de Desempenho

In [237]:
from sklearn.metrics import root_mean_squared_error

In [238]:
predicted_target_teste = lgbm.predict(features_teste)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [239]:
root_mean_squared_error(predicted_target_teste, target_teste)

4050.946296364888

#### Validação Cruzada

In [240]:
from sklearn.model_selection import KFold, cross_val_score

In [241]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [242]:
model = lgb.LGBMRegressor(num_leaves = 15, max_depth = 3, learning_rate = 0.1, n_estimators = 60)

In [244]:
cross_val_result = cross_val_score(model, scaled_all_features, target, cv = kfold);

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000229 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 334
[LightGBM] [Info] Number of data points in the train set: 1204, number of used features: 11
[LightGBM] [Info] Start training from score 13010.342228
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [245]:
print(f"Coeficiente R² médio: {cross_val_result.mean()}")
print(f"Coeficiente R² inferior: {cross_val_result.mean() - cross_val_result.std()}")
print(f"Coeficiente R² superior: {cross_val_result.mean() + cross_val_result.std()}")

Coeficiente R² médio: 0.8574484096097168
Coeficiente R² inferior: 0.8180650507145069
Coeficiente R² superior: 0.8968317685049267


#### Conclusão

Ocorreu praticamente um empate técnico entre o LightGBM e o XGBoost.
Apesar disso, na métrica de desempenho, o LightGBM foi ligeiramente melhor usando scaled_all_features.

### CatBoost

In [249]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.4 MB/s eta 0:00:00


In [301]:
from catboost.core import CatBoostRegressor

In [309]:
catboost = CatBoostRegressor(iterations = 75, learning_rate = 0.10, depth = 7, random_state = 0)

In [310]:
catboost.fit(features_treino, target_treino);

0:	learn: 11096.7213541	total: 3.81ms	remaining: 282ms
1:	learn: 10370.8239612	total: 4.37ms	remaining: 160ms
2:	learn: 9755.7383836	total: 5.9ms	remaining: 142ms
3:	learn: 9187.2664872	total: 7.35ms	remaining: 130ms
4:	learn: 8656.3861320	total: 13.7ms	remaining: 192ms
5:	learn: 8171.8658700	total: 14.2ms	remaining: 163ms
6:	learn: 7735.7643300	total: 15.6ms	remaining: 151ms
7:	learn: 7327.5305135	total: 21ms	remaining: 176ms
8:	learn: 6971.1983763	total: 22.6ms	remaining: 166ms
9:	learn: 6669.5714673	total: 23ms	remaining: 149ms
10:	learn: 6413.4874399	total: 23.4ms	remaining: 136ms
11:	learn: 6158.3320446	total: 24.4ms	remaining: 128ms
12:	learn: 5952.3027175	total: 30.7ms	remaining: 147ms
13:	learn: 5783.4733560	total: 32.2ms	remaining: 140ms
14:	learn: 5637.5084818	total: 32.7ms	remaining: 131ms
15:	learn: 5477.2082258	total: 38.5ms	remaining: 142ms
16:	learn: 5360.5466676	total: 40ms	remaining: 137ms
17:	learn: 5235.5888753	total: 41.6ms	remaining: 132ms
18:	learn: 5133.7165491	t

In [311]:
catboost.score(features_treino, target_treino)

np.float64(0.8959742114451366)

In [312]:
catboost.score(features_teste, target_teste)

np.float64(0.8871886423839948)

#### Resultados

all_features = scaled_all_features

CatBoost:

- Score em Treino: 0.895 ~ 0.90

- Score em Teste: 0.887 ~ 0.89

#### Métricas de Desempenho

In [313]:
predicted_target_teste = catboost.predict(features_teste)

In [314]:
from sklearn.metrics import root_mean_squared_error

In [315]:
root_mean_squared_error(target_teste, predicted_target_teste)

4241.458105099225

#### Validação Cruzada

In [316]:
from sklearn.model_selection import KFold, cross_val_score

In [317]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 0)

In [318]:
modelo = CatBoostRegressor(iterations = 75, learning_rate = 0.10, depth = 7, random_state = 0)

In [319]:
resultado = cross_val_score(modelo, all_features, target, cv = kfold)

0:	learn: 11054.0644801	total: 6.72ms	remaining: 497ms
1:	learn: 10318.3546299	total: 7.32ms	remaining: 267ms
2:	learn: 9675.8384761	total: 9ms	remaining: 216ms
3:	learn: 9073.3132547	total: 10.9ms	remaining: 193ms
4:	learn: 8488.3645577	total: 13ms	remaining: 183ms
5:	learn: 8006.9822973	total: 13.7ms	remaining: 158ms
6:	learn: 7550.5914270	total: 15.2ms	remaining: 147ms
7:	learn: 7145.9845984	total: 16.5ms	remaining: 138ms
8:	learn: 6785.2262303	total: 18.1ms	remaining: 132ms
9:	learn: 6505.4242602	total: 19.6ms	remaining: 128ms
10:	learn: 6219.9556953	total: 21.3ms	remaining: 124ms
11:	learn: 5970.7969237	total: 22.3ms	remaining: 117ms
12:	learn: 5785.6949346	total: 23.6ms	remaining: 113ms
13:	learn: 5616.8586596	total: 24ms	remaining: 105ms
14:	learn: 5464.2324896	total: 25.5ms	remaining: 102ms
15:	learn: 5315.9453845	total: 27ms	remaining: 99.4ms
16:	learn: 5183.9543530	total: 28ms	remaining: 95.6ms
17:	learn: 5069.5659354	total: 29.1ms	remaining: 92ms
18:	learn: 4980.7712846	tota

In [320]:
print(f"Coeficiente R² médio: {resultado.mean()}")
print(f"Coeficiente R² inferior: {resultado.mean() - resultado.std()}")
print(f"Coeficiente R² superior: {resultado.mean() + resultado.std()}")

Coeficiente R² médio: 0.8483613861522021
Coeficiente R² inferior: 0.8063578667102363
Coeficiente R² superior: 0.8903649055941679


#### Conclusão

O XGBoost e o LightGBM foram superiores ao CatBoost.

### Conclusão Final

De todos os modelos que foram treinados, os que se saíram melhor foram XGBoost e LightGBM. Ambos conseguiram boas pontuações, mas em termos produtivos, é provável que o LightGBM seria melhor por consumir menos recursos computacionais devido às podas que faz internamente.